In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

In [3]:
# 🔹 1. Cargar y preprocesar el dataset
df = pd.read_excel("data\data_emisores_gases_efecto_invernadero.xlsx")

# Preprocesamiento: llenar valores nulos con 0 en columnas numéricas
columnas_numericas = [
    'DIOXIDO_DE_CARBONO_GGCO2',
    'METANO_GGCH4',
    'METANO_EQUIVALENTE_GGCO2EQ',
    'OXIDO_NITROSO_GGN2O',
    'OXIDO_NITROSO_EQUIVALENTE_GGCO2EQ',
    'EMISIONES_GEI_GGCO2EQ'
]
df[columnas_numericas] = df[columnas_numericas].fillna(0)

# Llenar valores nulos en columnas categóricas
df['SUBCATEGORIA'] = df['SUBCATEGORIA'].fillna('Sin especificar')
df['CATEGORIA'] = df['CATEGORIA'].fillna('Sin especificar')
df['FUENTE_DE_EMISION'] = df['FUENTE_DE_EMISION'].fillna('Sin especificar')

In [18]:
# Asegurar que la columna de valores no tenga ceros o NaN
df = df[df["EMISIONES_GEI_GGCO2EQ"] > 0]

# Obtener lista de años únicos con datos válidos
years = sorted(df["ANIO"].unique())

# Verificar que hay datos para al menos un año
if not years:
    raise ValueError("No hay datos disponibles para mostrar en el treemap.")

# Crear el Treemap inicial con el primer año válido
df_year = df[df["ANIO"] == years[0]]

fig_treemap = px.treemap(
    df_year,
    path=['SECTOR', 'CATEGORIA', 'SUBCATEGORIA'],
    values='EMISIONES_GEI_GGCO2EQ',
    color='EMISIONES_GEI_GGCO2EQ',
    color_continuous_scale='RdYlBu_r',
    title=f'Emisiones de GEI en el Año {years[0]}'
)

# Crear los botones del menú para cada año
botones = []
for year in years:
    df_filtered = df[df["ANIO"] == year]
    
    if df_filtered.empty:
        continue  # Si no hay datos para el año, lo omitimos
    
    # Crear un nuevo Treemap para cada año
    treemap_trace = px.treemap(
        df_filtered,
        path=['SECTOR', 'CATEGORIA', 'SUBCATEGORIA'],
        values='EMISIONES_GEI_GGCO2EQ',
        color='EMISIONES_GEI_GGCO2EQ',
        color_continuous_scale='RdYlBu_r'
    ).data[0]  # Extraer la traza del treemap

    botones.append(
        dict(
            label=str(year),
            method="update",
            args=[{"type": "treemap", "parents": [treemap_trace.parents], 
                   "labels": [treemap_trace.labels], 
                   "values": [treemap_trace.values], 
                   "ids": [treemap_trace.ids]},
                  {"title": f"Emisiones de GEI en el Año {year}"}]
        )
    )

# Agregar el menú desplegable con los años disponibles
fig_treemap.update_layout(
    updatemenus=[dict(
        buttons=botones,
        direction="down",
        showactive=True,
        x=1.15,  # Ajusta la posición para alinear con el texto
        y=1.15
    )],
    annotations=[
        dict(
            text="Escoger año:",  # Texto que aparecerá junto al botón
            x=1.05,  # Posición horizontal
            y=1.15,  # Posición vertical (igual que el dropdown)
            xref="paper",  # Referencia relativa al gráfico
            yref="paper",
            showarrow=False,  # Sin flecha
            font=dict(size=14)  # Tamaño de la fuente
        )
    ],
    height=700
)

# Mostrar el gráfico interactivo
fig_treemap.show()

In [39]:
# Asegurar que la columna de valores no tenga ceros o NaN
df = df[df["EMISIONES_GEI_GGCO2EQ"] > 0]

# Obtener lista de años únicos con datos válidos
years = sorted(df["ANIO"].unique())

# Verificar que hay datos para al menos un año
if not years:
    raise ValueError("No hay datos disponibles para mostrar en el gráfico.")

# Crear el gráfico Sunburst inicial con el primer año válido
df_year = df[df["ANIO"] == years[0]]

fig_sunburst = px.sunburst(
    df_year,
    path=['SECTOR', 'CATEGORIA', 'FUENTE_DE_EMISION'],
    values='EMISIONES_GEI_GGCO2EQ',
    color='EMISIONES_GEI_GGCO2EQ',
    color_continuous_scale='Viridis',
    maxdepth=3
)

# Crear los botones del menú para cada año
botones = []
for year in years:
    df_filtered = df[df["ANIO"] == year]

    if df_filtered.empty:
        continue  # Si no hay datos para el año, lo omitimos

    # Extraer la traza del gráfico Sunburst para el año específico
    sunburst_trace = px.sunburst(
        df_filtered,
        path=['SECTOR', 'CATEGORIA', 'FUENTE_DE_EMISION'],
        values='EMISIONES_GEI_GGCO2EQ',
        color='EMISIONES_GEI_GGCO2EQ',
        color_continuous_scale='Viridis'
    ).data[0]  # Extraer la traza del gráfico

    # Agregar la opción al menú de selección de año
    botones.append(
        dict(
            label=str(year),
            method="update",
            args=[{"type": "sunburst",
                   "parents": [sunburst_trace.parents], 
                   "labels": [sunburst_trace.labels], 
                   "values": [sunburst_trace.values], 
                   "ids": [sunburst_trace.ids]},
                  {"title": f"Análisis Jerárquico de Emisiones de GEI en {year}"}]
        )
    )

# Agregar el menú desplegable con los años disponibles
fig_sunburst.update_layout(
    updatemenus=[dict(
        buttons=botones,
        direction="down",
        showactive=True,
        x=0.22,
        y=1.15
    )],annotations=[
        dict(
            text="Escoger año:",  # Texto que aparecerá junto al botón
            x=0,  # Posición horizontal
            y=1.15,  # Posición vertical (igual que el dropdown)
            xref="paper",  # Referencia relativa al gráfico
            yref="paper",
            showarrow=False,  # Sin flecha
            font=dict(size=14)  # Tamaño de la fuente
        )
    ],
    title={
        'text': f'Análisis Jerárquico de Emisiones de GEI en {years[0]}',
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20, 'color': '#333333'}
    },
    height=800
)

# Personalizar el hover
fig_sunburst.update_traces(
    hovertemplate="""
    <b>%{label}</b><br>
    Emisiones: %{value:,.2f} GgCO₂eq<br>
    Porcentaje: %{customdata[0]:.2f}%
    <extra></extra>
    """,
    textinfo='label+percent entry'
)

# Mostrar el gráfico
fig_sunburst.show()

In [18]:
# 3. Gráfica temporal
# Preparar datos agregados por año
temporal = df.groupby('ANIO').agg({
    'DIOXIDO_DE_CARBONO_GGCO2': 'sum',
    'METANO_EQUIVALENTE_GGCO2EQ': 'sum',
    'OXIDO_NITROSO_EQUIVALENTE_GGCO2EQ': 'sum'
}).reset_index()

fig_temporal = go.Figure()

# Añadir líneas para cada tipo de emisión
fig_temporal.add_trace(go.Scatter(
    x=temporal['ANIO'],
    y=temporal['DIOXIDO_DE_CARBONO_GGCO2'],
    name='CO2',
    mode='lines+markers'
))

fig_temporal.add_trace(go.Scatter(
    x=temporal['ANIO'],
    y=temporal['METANO_EQUIVALENTE_GGCO2EQ'],
    name='CH4 (CO2eq)',
    mode='lines+markers'
))

fig_temporal.add_trace(go.Scatter(
    x=temporal['ANIO'],
    y=temporal['OXIDO_NITROSO_EQUIVALENTE_GGCO2EQ'],
    name='N2O (CO2eq)',
    mode='lines+markers'
))

# Actualizar el diseño de la gráfica temporal
fig_temporal.update_layout(
    title='Evolución Temporal de Emisiones por Tipo de Gas',
    xaxis_title='Año',
    yaxis_title='Emisiones (GgCO2eq)',
    height=500,
    hovermode='x unified'
)
fig_temporal.show()